In [31]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [32]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('cora','splits/cora_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [33]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "trained_epochs/cora5.pt"
GNNcora=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.6).to(device)
GNNcora.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [34]:
#prendo l'embedding dalla rete
GNNcora.eval()
out = GNNcora.emb(list_mat, layer_norm)
#torch.save(out,'cora\emb_cora25.pt')


In [35]:
out.shape

torch.Size([2708, 448])

In [36]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

,source,target,weight
0,0,633,1
1,0,1862,1
2,0,2582,1
3,1,2,1
4,1,652,1
...,...,...,...
5273,2679,2682,1
5274,2681,2683,1
5275,2687,2688,1
5276,2694,2695,1


In [37]:
#definizione distanza manhattan
import numpy as np

def manhattan_distance(arr1, arr2):
    """
    Calculates the Manhattan distance between two arrays of the same length.
    """
    return np.sum(np.abs(arr1 - arr2))

In [38]:
'''#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('cora\cora_weights.csv', index = False)'''

"#aggiungo i pesi ottenuti dall'embedding\ndf_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])\nadj_mat['weight'] = adj_mat.apply(lambda x:\n                                    manhattan_distance(df_emb.loc[x['target']].values,\n                                    df_emb.loc[x['source']].values),\n                                    axis=1)\nadj_mat.to_csv('cora\\cora_weights.csv', index = False)"

In [39]:
#aggiungo i pesi ottenuti dall'embedding
from sklearn.metrics.pairwise import cosine_similarity

df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    cosine_similarity(np.array([df_emb.iloc[x['target']]]),
                                    np.array([df_emb.iloc[x['source']]]))[0][0],
                                    axis=1)
adj_mat.to_csv('weight_epochs/cora_cosine_weights5.csv', index = False)

In [ ]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('weight_epochs/cora_labels.csv', index = False)